### Object Detection on CPU (yolov3-tiny)

In [52]:
import numpy as np
import cv2
import time

# Load model
net = cv2.dnn.readNet('weights/yolov3-tiny.weights', 'cfg/yolov3-tiny.cfg')
layer_names = net.getLayerNames()
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers().ravel()]

# name of classes on which the model was trained on
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f]

# Load Video
cap = cv2.VideoCapture('../Videos/walking.mp4')

start_time = time.time()
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    frame_count += 1
    
    if ret:
        frame = cv2.resize(frame, (1600, 900))
        height, width, channels = frame.shape

        # Feed image to network
        blob = cv2.dnn.blobFromImage(image=frame, scalefactor=1/255.0, size=(416, 416), swapRB=True, crop=False)
        net.setInput(blob)
        outputs = net.forward(output_layers)

        # Bounding boxes
        boxes = []
        confidences = []
        class_ids = []

        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    # Rectangle co-ordinates
                    x = int(center_x - (w / 2))
                    y = int(center_y - (h / 2))

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        # Non-max suppression
        indices = cv2.dnn.NMSBoxes(bboxes=boxes, scores=confidences, score_threshold=0.5, nms_threshold=0.4)

        # Draw most confident boxes in the image        
        for i, box in enumerate(boxes):
            if i in indices:
                x, y, w, h = box
                label = str(classes[class_ids[i]])

                cv2.rectangle(frame, (x, y), (x+w, y+h), color=(255,255,0), thickness=2)
                text = "{}: {:.4f}".format(classes[class_ids[i]], confidences[i])
                cv2.putText(img=frame, text=text, org=(x, y-5), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,255,0), thickness=2)

        # Show output
        elapsed_time = time.time() - start_time
        fps = frame_count / elapsed_time
        cv2.putText(img=frame, text='FPS : ' + str(round(fps, 2)), org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0,255,255), thickness=2)
        
        cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()